In [49]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import math
import ipywidgets as widgets
from ipywidgets import VBox, HBox, interactive
import math
import matplotlib.image as img
import cv2
from IPython.display import clear_output, display
from tkinter import Tk, filedialog

This notebook consists of the following classes:

    class maze corner
    class basics
    class bottom_cam
    class gui
        class processing_gui
        class bc_gui
        class maze_corner_gui

    class side_cams
    class bottom_cams
    class defish
    class session
    class main

Name all of your .csv/.mp4 files strictly like this: 
                       210_F1-83_220518_OTT_bottom
                       Line_ID_Date_Paradigm_Camera                     

In [84]:
class main:
    path = "" 
    framerate = {}
    l_maze_corners_bc = []
    l_maze_corners_tc = []

    def __init__(self):
        gui.main_gui()
        
    def all_information_given(self):
        self.l_sessions = [session(session_ID, self.bottom_cam_used, self.top_cam_used, self.side_cam_used) for session_ID in [elem[0:-11] for elem in os.listdir(self.path) if elem.endswith('.csv')]]
        #self.l_sessions erzeugt sessionID, allerdings ist die Länge davon abhängig davon, welche cam verwendet wurde!
        if self.bottom_cam_used & self.all_files_there:
            main.l_maze_corners_bc = [session.bc.mc for session in main.l_sessions]
        if self.top_cam_used & self.all_files_there:
            main.l_maze_corners_tc = [session.tc.mc for session in main.l_sessions]
        if self.side_cam_used:
            pass
        if self.top_cam_used or self.bottom_cam_used:
            gui.get_maze_corner_gui()
        
    def get_maze_corners(self):
        self.l_maze_corners = self.l_maze_corners_bc + self.l_maze_corners_tc
        gui.maze_corner_gui()
            
    def get_clean_df_gui(self):
        if self.bottom_cam_used:
            gui.clean_df_gui("bottom_cam")
        if self.top_cam_used:
            gui.clean_df_gui("top_cam")
        if self.side_cam_used:
            pass
                   
    def get_processed_dfs(self, DLC_likelihood_threshold, cam):
        self.dict_bodyparts = {}
        if cam == "bottom_cam":
            #processed_dfs nicht in main_liste setzen, sondern zu den session objects
            for session in self.l_sessions:
                session.bc.processed_df = clean_df(session.bc.df, session.bc.mc.results, self.framerate["bottom_cam"], DLC_likelihood_threshold, cam, self.mazelength).df 
        elif cam =="top_cam":
            for session in self.l_sessions:
                session.tc.processed_df = clean_df(session.tc.df, session.tc.mc.results, self.framerate["top_cam"], DLC_likelihood_threshold, cam, self.mazelength).df
        elif cam == "side_cam":
            pass        
        gui.select_functions()
        
    #vor execute functions erst andere cams analysieren
    def execute_functions(self, dict_selected_functions):
        #analysed dfs wie für processed dfs verfahren
        for session in self.l_sessions:
            session.analyzed_df = df_functions(session, dict_selected_functions).df_bc

In [85]:
class session(main):
    def __init__(self, session_ID, bottom_cam_used, top_cam_used, side_cam_used):
        self.bottom_cam_used = bottom_cam_used
        self.top_cam_used = top_cam_used
        self.side_cam_used = side_cam_used
        self.session_ID = session_ID
        #####[slicing range] muss angepasst werden an ending (bottom, top, side)
        bc_csv = self.session_ID in set([elem[0:-11] for elem in os.listdir(main.path) if elem.endswith('_bottom.csv')])
        bc_mp4 = self.session_ID in set([elem[0:-11] for elem in os.listdir(main.path) if elem.endswith('_bottom.mp4')]) 
        tc_csv = self.session_ID in set([elem[0:-8] for elem in os.listdir(main.path) if elem.endswith('_top.csv')])
        tc_mp4 = self.session_ID in set([elem[0:-8] for elem in os.listdir(main.path) if elem.endswith('_top.mp4')])
        sc_csv = self.session_ID in set([elem[0:-9] for elem in os.listdir(main.path) if elem.endswith('_side.csv')])
        sc_mp4 = self.session_ID in set([elem[0:-9] for elem in os.listdir(main.path) if elem.endswith('_side.mp4')])
        
        main.all_files_there = True
        missing_files_statement = "\nPlease name your files like as the following: 210_F1-83_220518_OTT_bottom.mp4 \nLine_ID_Date_Paradigm_Camera_ending"
        if bottom_cam_used:
            if bc_csv == False:
                print("Missing _bottom.csv file for session {} in {}!".format(self.session_ID, main.path), missing_files_statement)
                main.all_files_there = False
            elif bc_mp4 == False:
                print("Missing _bottom.mp4 file for session {} in {}!".format(self.session_ID, main.path), missing_files_statement)
                main.all_files_there = False
            else:
                bc_video_filename = session_ID + "_bottom.mp4"
                bc_csv_filename = session_ID + "_bottom.csv"
                bc_df = pd.read_csv(main.path + bc_csv_filename, skiprows=1, index_col=0, header=[0, 1])
                self.bc = bottom_cam(self.session_ID, bc_df)

        if top_cam_used:
            if tc_csv ==False:
                print("Missing _top.csv file for session {} in {}!".format(self.session_ID, main.path), missing_files_statement)
                main.all_files_there = False
            elif tc_mp4 == False:
                print("Missing top.mp4 file for session {} in {}!".format(self.session_ID, main.path), missing_files_statement)
                main.all_files_there = False
            else:  
                tc_video_filename = session_ID + "_top.mp4"
                tc_csv_filename = session_ID + "_top.csv"
                tc_df = pd.read_csv(main.path + tc_csv_filename, skiprows=1, index_col=0, header=[0, 1])
                self.tc = top_cam(self.session_ID, tc_df)
            
        if side_cam_used:
            if sc_csv == False:
                print("Missing _side.csv file for session {} in {}!".format(self.session_ID, main.path), missing_files_statement)
                main.all_files_there = False
            elif sc_mp4 == False:
                print("Missing side.mp4 file for session {} in {}!".format(self.session_ID, main.path),missing_files_statement)
                main.all_files_there = False
            else:
                sc_video_filename = session_ID + "_side.mp4"
                sc_csv_filename = session_ID + "_side.csv"
                sc_df = pd.read_csv(main.path + sc_csv_filename, skiprows=1, index_col=0, header=[0, 1])
                #create side_cam object
        
        
    """def get_session_averages(df):#?????
        freezing_bout_count = df[('Freezing_bout', 'count')].unique().shape[0] - 1

        if frezing_bout_count > 0:
            df[('whole_session', 'percentage_time_spent_freezing')] = (df.loc[df[('Freezing_bout', '')] == True].shape[0] / df.shape[0]) * 100
            df[('whole_session', 'median_freezing_bout_duration')] = np.nanmedian(df[('Freezing_bout', 'duration')].unique())
            df[('whole_session', 'median_x_norm_cm_all_freezing_bouts')] = np.nanmedian(df[('Freezing_bout', 'mean_x_norm_cm')].unique())
            df[('whole_session', 'median_y_norm_cm_all_freezing_bouts')] = np.nanmedian(df[('Freezing_bout', 'mean_y_norm_cm')].unique())
        else:
            df[('whole_session', 'percentage_time_spent_freezing')] = 0
            df[('whole_session', 'median_freezing_bout_duration')] = np.NaN
            df[('whole_session', 'median_x_norm_cm_all_freezing_bouts')] = np.NaN
            df[('whole_session', 'median_y_norm_cm_all_freezing_bouts')] = np.NaN       

        gait_disruption_bout_count = df[('GaitDisruption_bout', 'count')].unique().shape[0] - 1

        if gait_disruption_bout_count > 0:
            df[('whole_session', 'percentage_time_spent_gait_disrupted')] = (df.loc[df[('GaitDisruption_bout', '')] == True].shape[0] / df.shape[0]) * 100
            df[('whole_session', 'median_gait_disruption_bout_duration')] = np.nanmedian(df[('GaitDisruption_bout', 'duration')].unique())
            df[('whole_session', 'median_x_norm_cm_all_gait_disruption_bouts')] = np.nanmedian(df[('GaitDisruption_bout', 'mean_x_norm_cm')].unique())
            df[('whole_session', 'median_y_norm_cm_all_gait_disruption_bouts')] = np.nanmedian(df[('GaitDisruption_bout', 'mean_y_norm_cm')].unique())
        else:
            df[('whole_session', 'percentage_time_spent_gait_disrupted')] = 0
            df[('whole_session', 'median_gait_disruption_bout_duration')] = np.NaN
            df[('whole_session', 'median_x_norm_cm_all_gait_disruption_bouts')] = np.NaN
            df[('whole_session', 'median_y_norm_cm_all_gait_disruption_bouts')] = np.NaN

        return df"""

In [86]:
class bottom_cam(session):
    def __init__(self, session_ID, df):
        self.session_ID = session_ID
        self.df = df
        self.mc = maze_corners(self.path, session_ID + "_bottom.mp4")

In [87]:
class top_cam:
    def __init__(self):
        self.session_ID = session_ID
        self.df = df
        self.mc = maze_corners(self.path, session_ID + "_top.mp4")

In [88]:
class side_cam:
    def __init__(self):
        pass
        #stitch()
        

In [89]:
class clean_df(session): #get_time, identify duplicates, exclude_frames, rotate, translate, normalize
    def __init__(self, df, results, framerate, threshold, cam, mazelength):
        #side cams geben keine results weil sie kein mazecorner objekt haben
        self.threshold = threshold
        self.framerate = framerate
        self.results = results
        self.l_bodyparts = [elem[0] for elem in df.columns[::3]]
        main.dict_bodyparts[cam] = self.l_bodyparts
        df = self.get_time(df)
        df = self.identify_duplicates(df)
        df[('all', 'exclude')] = False
        df = self.exclude_frames(df)
        if "CenterOfGravity" not in self.l_bodyparts:
            df = self.get_center_of_gravity(df)
        df = self.normalize_coordinates(df)
        self.df = df
        self.mazelength = mazelength
        
    def get_time(self, df):
        df['time'] = np.NaN
        df['time'] = df['EarRight'].index/self.framerate

        return df
        # in future version: check for NaN
        
    def identify_duplicates(self, df):
        l_indices = list(df.index)
        l_unique_indices = list(set(l_indices))

        if len(l_indices) != len(l_unique_indices):
            l_duplicates = []
            for index in l_unique_indices:
                if l_indices.count(index) > 1:
                    l_duplicates.append(index)
            df.loc[l_duplicates, ('all', 'exclude')] = True

        return df

    def exclude_frames(self, df):
        for bodypart in self.l_bodyparts:
            df.loc[:, (bodypart, 'exclude')] = False
            df.loc[df[bodypart]['likelihood'] < self.threshold, (bodypart, 'exclude')] = True
            df.loc[df[('all', 'exclude')] == True, (bodypart, 'exclude')] = True
        return df
    
    def rotate(self, xy, theta):
        cos_theta, sin_theta = math.cos(theta), math.sin(theta)

        return (
            xy[0] * cos_theta - xy[1] * sin_theta,
            xy[0] * sin_theta + xy[1] * cos_theta
        )

    def translate(self, xy, offset):
        return xy[0] + offset[0], xy[1] + offset[1]


    def normalize_coordinates(self, df):##### changed it!!! 
        # Extract all reference information - ???

        length = self.results['length']
        width = self.results['width']
        offset_to_standard = (-self.results['offset_x'], -self.results['offset_y'])
        offset_from_standard = (self.results['offset_x'], self.results['offset_y'])
        theta_to_standard = -self.results['theta']


        length_in_px = length
        cm_per_px = self.maze_length_in_cm/length_in_px

        for bodypart in self.l_bodyparts: #removed + ['CenterOfGravity'] since we will label it in DLC
            df[(bodypart, 'x_norm')] = self.translate(self.rotate((df[(bodypart,'x')], df[(bodypart,'y')]), theta_to_standard), offset_to_standard)[0]
            df[(bodypart, 'y_norm')] = self.translate(self.rotate((df[(bodypart,'x')], df[(bodypart,'y')]), theta_to_standard), offset_to_standard)[1]
            df[(bodypart, 'x_norm_cm')] = 3 - (df[(bodypart, 'x_norm')] * cm_per_px)
            df[(bodypart, 'y_norm_cm')] = 50 - (df[(bodypart, 'y_norm')] * cm_per_px)

        return df    

    
    def get_center_of_gravity(self, df):
        df.loc[(df[('all', 'exclude')] == False) & (df[('EarRight', 'exclude')] == False) & (df[('EarLeft', 'exclude')] == False) & (df[('TailBase', 'exclude')] == False), 
           ('CenterOfGravity', 'x')] = (df.loc[df[('all', 'exclude')] == False, ('EarRight', 'x')] + 
                                          df.loc[df[('all', 'exclude')] == False, ('EarLeft', 'x')] + 
                                          df.loc[df[('all', 'exclude')] == False, ('TailBase', 'x')]) / 3

        df.loc[(df[('all', 'exclude')] == False) & (df[('EarRight', 'exclude')] == False) & (df[('EarLeft', 'exclude')] == False) & (df[('TailBase', 'exclude')] == False), 
           ('CenterOfGravity', 'y')] = (df.loc[df[('all', 'exclude')] == False, ('EarRight', 'y')] + 
                                          df.loc[df[('all', 'exclude')] == False, ('EarLeft', 'y')] + 
                                          df.loc[df[('all', 'exclude')] == False, ('TailBase', 'y')]) / 3

        df[('CenterOfGravity', 'exclude')] = False
        df.loc[(df[('CenterOfGravity', 'x')].isnull()) | (df[('CenterOfGravity', 'y')].isnull()), ('CenterOfGravity', 'exclude')] = True

        self.l_bodyparts.append('CenterOfGravity')
        return df

In [90]:
class df_functions(bottom_cam, top_cam, side_cam):
    immobility_threshold = 16

    min_freezing_duration = 1

    TIME_OF_GAIT_BEFORE_DISRUPT = 0.5
    TARGET_TIME_GAIT_DISRUPTION = 0.2
    
    def __init__(self, session, dict_selected_functions):
        self.session = session
        self.dict_selected_functions = dict_selected_functions
        if main.bottom_cam_used:
            df = session.bc.processed_df
            self.l_bodyparts = main.dict_bodyparts["bottom_cam"]
            self.framerate = main.framerate_bc
            
            #if schleife für dict setzen
            df = self.get_speed_and_rolling_speed(df)
            df = self.get_immobility(df)
            df = self.get_freezing_bouts(df)
            df = self.get_direction_bc(df)
            df = self.get_gait_disruption_bouts(df)
            self.df_bc = df
            
        if main.top_cam_used:
            self.l_bodyparts = main.dict_bodyparts["top_cam"]
            df = session.tc.processed_df
            self.l_bodyparts = main.dict_bodyparts["top_cam"]
            self.framerate = main.framerate_tc
            
            #if schleife für dict setzen
            df = self.get_speed_and_rolling_speed(df)
            df = self.get_immobility(df)
            df = self.get_freezing_bouts(df)
            df = self.get_direction_bc(df)
            df = self.get_gait_disruption_bouts(df)
            self.df_bc = df
            
        if main.side_cam_used:
            self.l_bodyparts = main.dict_bodyparts["side_cam"]
        
        #wenn die weiteren Kameras implementiert sind alle dfs zu einem großen analyzed_df fusionieren
        
    
    def get_speed_and_rolling_speed(self, df):
        for bodypart in self.l_bodyparts:
            df[(bodypart, 'speed_px_per_s')] = np.NaN
            df[(bodypart, 'rolling_speed_px_per_s')] = np.NaN

            # Limitation: since we have to exclude some frames, these calculations are not made frame by frame (yet for most)
            df.loc[(df[('all', 'exclude')] == False) & (df[(bodypart, 'exclude')] == False), (bodypart, 'speed_px_per_s')] = (((df.loc[(df[('all', 'exclude')] == False) & (df[(bodypart, 'exclude')] == False), (bodypart, 'x')].diff()**2                                                                                                        + df.loc[(df[('all', 'exclude')] == False) & (df[(bodypart, 'exclude')] == False), (bodypart, 'y')].diff()**2)**(1/2)) 
                                                                                                                             / df.loc[(df[('all', 'exclude')] == False) & (df[(bodypart, 'exclude')] == False), 'time'].diff())
            df.loc[(df[('all', 'exclude')] == False) & (df[(bodypart, 'exclude')] == False), (bodypart, 'rolling_speed_px_per_s')] = df.loc[df[('all', 'exclude')] == False, (bodypart, 'speed_px_per_s')].rolling(5, min_periods=3, center=True).mean()

        return df
    
    #checken, dass CenterOfGravity in l_bodyparts ist, ansonsten calculate function hinzufügen? 
    def get_direction_bc(self, df): #used Snout instead of EarLeft & EarRight (less secure parameter?, but better suitable for BottomCam?)
            #could also synchronize with top Cam data and use direction from there
        df[('moving_towards_maze_end', '')] = False
        df.loc[(df[('Snout', 'y_norm')] < df[('TailBase', 'y_norm')]), ('moving_towards_maze_end', '')] = True
        #df.loc[(df[('Snout', 'y_norm')] < df[('CenterOfGravity', 'y_norm')]), ('moving_towards_maze_end', '')] = True
        return df
    
    def get_dircetion_tc(self, df):
        pass
    
    def get_direction_sc(self, df):
        pass
    
    def get_immobility(self, df):
        for bodypart in self.l_bodyparts:
            # create 'immobility' column and set base value to false
            df.loc[ :, (bodypart, 'immobility')] = False
            df.loc[df[(bodypart,'rolling_speed_px_per_s')] < self.immobility_threshold, (bodypart, 'immobility')] = True
        return df
        
    def get_gait_disruption_bouts(self, df):
        df[('GaitDisruption_bout', '')] = False
        df[('GaitDisruption_bout', 'count')] = np.NaN
        df[('GaitDisruption_bout', 'duration')] = np.NaN
        df[('GaitDisruption_bout', 'mean_x_norm_cm')] = np.NaN
        df[('GaitDisruption_bout', 'mean_y_norm_cm')] = np.NaN
        df[('GaitDisruption_bout', 'direction_bool')] = ''
        df[('GaitDisruption_bout', 'direction_mean')] = np.NaN


        l_timesteps = []
        for i in range(self.framerate):
            l_timesteps.append(i/self.framerate)

        time_gait_disruption = self.find_nearest(np.asarray(l_timesteps), self.TARGET_TIME_GAIT_DISRUPTION)
        frames_difference = l_timesteps.index(time_gait_disruption)

        gait_disruption_threshold_reached = df.loc[df['all_freezing_bodyparts_immobile'] == True, 'time'].iloc[np.where(np.round(df.loc[df['all_freezing_bodyparts_immobile'] == True, 'time'].diff(frames_difference).values, 7) == round(time_gait_disruption, 7))[0]].values

        # could still throw an error if only a single frame, because of slicing to find first and last index of each interval?    
        if gait_disruption_threshold_reached.shape[0] > 0:
            indices = np.asarray(df.loc[df['time'].isin(gait_disruption_threshold_reached)].index)
            lower_end = (indices+1)[:-1]
            upper_end = indices[1:]
            mask = lower_end < upper_end
            mask_last = np.concatenate([mask, np.array([True])])
            mask_first = np.concatenate([np.array([True]), mask])

            last_value_of_intervals = indices[mask_last]
            first_value_of_intervals = indices[mask_first]
            
            first_value_of_intervals = first_value_of_intervals-l_timesteps.index(self.find_nearest(np.asarray(l_timesteps), self.TARGET_TIME_GAIT_DISRUPTION))

            interval_ranges = np.column_stack([first_value_of_intervals,last_value_of_intervals])

            frames_prior_to_interval_start = l_timesteps.index(self.find_nearest(np.asarray(l_timesteps), self.TIME_OF_GAIT_BEFORE_DISRUPT))
            bout_count = 0

            if interval_ranges.shape[0] > 0:
                for first_idx, last_idx in interval_ranges:
                    start_idx_gait_check = first_idx - frames_prior_to_interval_start
                    if df.loc[start_idx_gait_check:first_idx-1, 'all_freezing_bodyparts_immobile'].any() == False:
                        bout_count = bout_count + 1
                        bout_duration = (df.loc[last_idx, 'time'] - df.loc[first_idx, 'time'])[0]
                        mean_pos_x_norm_cm = df.loc[first_idx:last_idx, ('CenterOfGravity', 'x_norm_cm')].mean()
                        mean_pos_y_norm_cm = df.loc[first_idx:last_idx, ('CenterOfGravity', 'y_norm_cm')].mean()
                        direction_bool = df.loc[first_idx:last_idx, ('moving_towards_maze_end', '')].all()
                        direction_mean = df.loc[first_idx:last_idx, ('moving_towards_maze_end', '')].mean()

                        df.loc[first_idx:last_idx, ('GaitDisruption_bout', '')] = True
                        df.loc[first_idx:last_idx, ('GaitDisruption_bout', 'count')] = bout_count
                        df.loc[first_idx:last_idx, ('GaitDisruption_bout', 'duration')] = bout_duration
                        df.loc[first_idx:last_idx, ('GaitDisruption_bout', 'mean_x_norm_cm')] = mean_pos_x_norm_cm
                        df.loc[first_idx:last_idx, ('GaitDisruption_bout', 'mean_y_norm_cm')] = mean_pos_y_norm_cm
                        df.loc[first_idx:last_idx, ('GaitDisruption_bout', 'direction_bool')] = direction_bool
                        df.loc[first_idx:last_idx, ('GaitDisruption_bout', 'direction_mean')] = direction_mean

        return df
        
    def find_nearest(self, array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return array[idx]

    def get_freezing_bouts(self, df):
        df[('Freezing_bout', '')] = False
        df[('Freezing_bout', 'count')] = np.NaN
        df[('Freezing_bout', 'duration')] = np.NaN
        df[('Freezing_bout', 'mean_x_norm_cm')] = np.NaN
        df[('Freezing_bout', 'mean_y_norm_cm')] = np.NaN

        df['all_freezing_bodyparts_immobile'] = df[[('EarRight', 'immobility'), ('EarLeft', 'immobility'), ('CenterOfGravity', 'immobility')]].all(axis=1)

        times_where_freezing_threshold_was_reached = df.loc[df['all_freezing_bodyparts_immobile'] == True, 'time'].iloc[np.where(df.loc[df['all_freezing_bodyparts_immobile'] == True, 'time'].diff(self.framerate).values == 1)[0]].values

        # could still throw an error if only a single frame, because of slicing to find first and last index of each interval?
        if times_where_freezing_threshold_was_reached.shape[0] > 0:
            indices = np.asarray(df.loc[df['time'].isin(times_where_freezing_threshold_was_reached)].index)
            lower_end = (indices+1)[:-1]
            upper_end = indices[1:]
            mask = lower_end < upper_end
            mask_last = np.concatenate([mask, np.array([True])])
            mask_first = np.concatenate([np.array([True]), mask])

            last_value_of_intervals = indices[mask_last]
            first_value_of_intervals = indices[mask_first]

            first_value_of_intervals = first_value_of_intervals-self.self.framerate

            interval_ranges = np.column_stack([first_value_of_intervals,last_value_of_intervals])

            bout_count = 0

            if interval_ranges.shape[0] > 0:
                for first_idx, last_idx in interval_ranges:
                        bout_count = bout_count + 1
                        bout_duration = (df.loc[last_idx, 'time'] - df.loc[first_idx, 'time'])[0]
                        mean_pos_x_norm_cm = df.loc[first_idx:last_idx, ('CenterOfGravity', 'x_norm_cm')].mean()
                        mean_pos_y_norm_cm = df.loc[first_idx:last_idx, ('CenterOfGravity', 'y_norm_cm')].mean()

                        df.loc[first_idx:last_idx, ('Freezing_bout', '')] = True
                        df.loc[first_idx:last_idx, ('Freezing_bout', 'count')] = bout_count
                        df.loc[first_idx:last_idx, ('Freezing_bout', 'duration')] = bout_duration
                        df.loc[first_idx:last_idx, ('Freezing_bout', 'mean_x_norm_cm')] = mean_pos_x_norm_cm
                        df.loc[first_idx:last_idx, ('Freezing_bout', 'mean_y_norm_cm')] = mean_pos_y_norm_cm

        return df
    
    
    
    
    #Funktionen für Inken:
    
    def get_bodypart_angles(self, df):
        df['angle_secondtoe_hindpawright'] = np.NaN
        df['angle_secondtoe_hindpawleft'] = np.NaN
        df['angle_hindpawright_kneeright'] = np.NaN
        df['angle_hindpawleft_kneeleft'] = np.NaN
        df['angle_kneeright_bodyaxis'] = np.NaN
        df['angle_kneeleft_bodyaxis'] = np.NaN
        #Mittelzeh, Pfotenmitte, Knie, Körperachse (TailBase to Center of Gravity)
    
    def get_distance_secondtoe_pawbase(self, df):
        df['distance_secondtoe_hindpawright'] = np.NaN
        df['distance_secondtoe_hindpawleft'] = np.NaN
        
    def get_paw_area(self, df):
        df['area_hindpawright'] = np.NaN
        df['area_hindpawleft'] = np.NaN
    
    
    
    

In [91]:
class maze_corners(bottom_cam, top_cam):    
    def __init__(self, path, filename):
        self.filename = filename
        self.filepath = main.path + filename
        self.cap = cv2.VideoCapture(self.filepath)
        self.ret, self.frame = self.cap.read()
        self.results = {}

In [92]:
class gui():
    class main_gui(main):
        def __init__(self):             
            self.path = ""
            folder_select = widgets.Button(description="Select folder")
            folder_select.on_click(self.select_folder)
            
            select_recording_modalities = widgets.Label(value="Select recording modalities")
            self.bottom_cam_check = widgets.Checkbox(
                        value=False,
                        description='Bottom Cam',
                        disabled=False,
                        indent=False)
            self.top_cam_check = widgets.Checkbox(
                        value=False,
                        description='Top Cam',
                        disabled=False,
                        indent=False)
            self.side_cam_check = widgets.Checkbox(
                        value=False,
                        description='Side Cam',
                        disabled=False,
                        indent=False)
            
            confirm_button = widgets.Button(description = "Confirm Settings")
            confirm_button.on_click(self.confirm_settings)
            
            col0 = VBox([folder_select])
            col1 = VBox([select_recording_modalities, self.bottom_cam_check, self.top_cam_check, self.side_cam_check])
            col2 = VBox([confirm_button])
            box = HBox([col0, col1, col2])
            display(box)

        def select_folder(self, b):
            root = Tk()
            root.withdraw()
            root.call('wm', 'attributes', '.', '-topmost', True)
            self.path = filedialog.askdirectory() + "//"
            display(self.path)        
            
        def confirm_settings(self, b):
            if self.path == "":
                display("Set the path before continuing!")
            else:
                main.bottom_cam_used = self.bottom_cam_check.value
                main.top_cam_used = self.top_cam_check.value
                main.side_cam_used = self.side_cam_check.value
                main.path = self.path
                main.all_information_given(main)
        
    class bc_gui(main):
        def __init__(self):
            #defish
            pass
                
    class tc_gui(main):
        def __init__(self):
            pass
    
    class get_maze_corner_gui(main):
        def __init__(self):
            maze_corners_button = widgets.Button(description = "Set maze corners")
            maze_corners_button.on_click(main.get_maze_corners(main))
            
            row0 = HBox([maze_corners_button])
            box = VBox([row0])
            
            display(box)
    
    class maze_corner_gui(main):
        def __init__(self):
            self.maze_corner_idx = 0
            self.actualize()
            self.create_gui()

        def on_load_next_button_click(self, b):
            if self.results_saved:
                if self.maze_corner_idx >= (len(main.l_maze_corners)-1):
                    print("Maze Corners for all videos set.")
                    #check, whether all Maze Corners are saved to bc.mc.results
                    main.get_clean_df_gui(main)
                else: 
                    self.maze_corner_idx += 1
                    self.actualize()
            else:
                print("Please save the settings before continuing!")

        def on_load_previous_button_click(self, b):
            if self.maze_corner_idx <= 0:
                print("Index out of range! Index has been set to 0.")
                self.maze_corner_idx = 0
            else:
                if self.results_saved:
                    self.maze_corner_idx -= 1
                    self.actualize()
                else:
                    print("Please save the settings before continuing!")

        def actualize(self):
            self.filename = main.l_maze_corners[self.maze_corner_idx].filename
            self.filepath = main.l_maze_corners[self.maze_corner_idx].filepath
            self.cap = cv2.VideoCapture(self.filepath)
            self.ret, self.frame = self.cap.read()
            self.results_saved = False

        def on_save_button_click(self, b):
            main.l_maze_corners[self.maze_corner_idx].results["offset_x"] = self.interactive_plot.children[0].value
            main.l_maze_corners[self.maze_corner_idx].results["offset_y"] = self.interactive_plot.children[1].value
            main.l_maze_corners[self.maze_corner_idx].results["length"] = self.interactive_plot.children[2].value
            main.l_maze_corners[self.maze_corner_idx].results["width"] = self.interactive_plot.children[3].value
            main.l_maze_corners[self.maze_corner_idx].results["theta"] = math.radians(self.interactive_plot.children[4].value)
            self.results_saved = True
            # Save the results:??????????????????????????????????????????????????????????
            #with open('reference_coordinates.p', 'wb') as fp:
            #    pickle.dump(d_results, fp, protocol=pickle.HIGHEST_PROTOCOL)


        def create_gui(self):
            width, height = self.frame.shape[0], self.frame.shape[1]#replace slider with int, since slider are very slow
            slider_x = widgets.IntSlider(value=300, min=0, max=width, step=1, description='x offset')
            slider_y = widgets.IntSlider(value=5, min=0, max=height, step=1, description='y offset')
            slider_length = widgets.IntSlider(value=300, min=0, max=height, step=1)
            slider_width = widgets.IntSlider(value=20, min=0, max=width, step=1)
            slider_degrees = widgets.FloatSlider(value=0, min=0, max=90, step=0.1)

            self.interactive_plot = interactive(main.l_maze_corners[self.maze_corner_idx].f, x=slider_x, y=slider_y, length=slider_length, width=slider_width, degrees=slider_degrees)

            self.interactive_plot.children[-1].layout.height = '600px'

            load_next_button = widgets.Button(description="Load next file")
            save_button = widgets.Button(description="Save settings")
            load_previous_button = widgets.Button(description="Load previous file")

            load_next_button.on_click(self.on_load_next_button_click)
            load_previous_button.on_click(self.on_load_previous_button_click)
            save_button.on_click(self.on_save_button_click)

            col0 = VBox([load_next_button, save_button])
            col1 = VBox([self.interactive_plot.children[0], self.interactive_plot.children[1]])
            col2 = VBox([self.interactive_plot.children[2], self.interactive_plot.children[3]])
            col3 = VBox([self.interactive_plot.children[4], load_previous_button])
            box = HBox([col0, col1, col2, col3])
            
            display(box, self.interactive_plot.children[-1])
            
        
        def rotate(self, xy, theta):
            cos_theta, sin_theta = math.cos(theta), math.sin(theta)

            return (
                xy[0] * cos_theta - xy[1] * sin_theta,
                xy[0] * sin_theta + xy[1] * cos_theta
            )

        def translate(self, xy, offset):
            return xy[0] + offset[0], xy[1] + offset[1]

        def f(self, x, y, length, width, degrees):
            offset = (x, y)
            corners = [(0, 0), (width, 0), (width, length), (0, length)]
            rotated_and_shifted_corners = [self.translate(self.rotate(xy, math.radians(degrees)), offset) for xy in corners]

            end_right_corner = list(rotated_and_shifted_corners[0]) + ['red']
            end_left_corner = list(rotated_and_shifted_corners[1]) + ['orange']
            start_left_corner = list(rotated_and_shifted_corners[2]) + ['cyan']
            start_right_corner = list(rotated_and_shifted_corners[3]) + ['green']

            fig = plt.figure(figsize=(18, 10))
            gs = fig.add_gridspec(2, 4)

            fig.add_subplot(gs[0:2, 0:2])
            plt.imshow(self.frame)
            plt.ylim(0,self.frame.shape[0])
            plt.xlim(0,self.frame.shape[1])

            if len(self.results.keys()) > 0:
                saved_current = 'saved'
            else:
                saved_current = 'missing'

            plt.title('current file: {} (analysis {})'.format(self.filename, saved_current))

            l_corners = [start_right_corner, start_left_corner, end_right_corner, end_left_corner]

            for corner in l_corners:
                plt.scatter(corner[0], corner[1], c=corner[2], s=100)

            fig.add_subplot(gs[0, 2])
            plt.imshow(self.frame)
            plt.scatter(l_corners[0][0], l_corners[0][1], c=l_corners[0][2], s=100)
            plt.xlim(l_corners[0][0]-25, l_corners[0][0]+25)
            plt.ylim(l_corners[0][1]-25, l_corners[0][1]+25)
            plt.title('start right corner')

            fig.add_subplot(gs[0, 3])
            plt.imshow(self.frame)
            plt.scatter(l_corners[1][0], l_corners[1][1], c=l_corners[1][2], s=100)
            plt.xlim(l_corners[1][0]-25, l_corners[1][0]+25)
            plt.ylim(l_corners[1][1]-25, l_corners[1][1]+25)
            plt.title('start left corner')

            fig.add_subplot(gs[1, 2])
            plt.imshow(self.frame)
            plt.scatter(l_corners[2][0], l_corners[2][1], c=l_corners[2][2], s=100)
            plt.xlim(l_corners[2][0]-25, l_corners[2][0]+25)
            plt.ylim(l_corners[2][1]-25, l_corners[2][1]+25)
            plt.title('end right corner')

            fig.add_subplot(gs[1, 3])
            plt.imshow(self.frame)
            plt.scatter(l_corners[3][0], l_corners[3][1], c=l_corners[3][2], s=100)
            plt.xlim(l_corners[3][0]-25, l_corners[3][0]+25)
            plt.ylim(l_corners[3][1]-25, l_corners[3][1]+25)
            plt.title('end left corner')

            plt.show()
            

    class clean_df_gui(bottom_cam, top_cam, side_cam):
        def __init__(self, cam):
            self.cam = cam
            self.clean_df = False
            clean_df_button = widgets.Button(description = "Clean Dataframe for {}!".format(self.cam))
            clean_df_button.on_click(self.get_processed_df)
            self.likelihood_threshold_slider = widgets.FloatSlider(min=0, max=1, steps=0.01, value=0.75, description = "Set the DLC likelihood threshold for {}!".format(self.cam))
            self.framerate_slider = widgets.IntSlider(min=0, max=200, value=30, description = "Set the Framerate for {}!".format(self.cam))
            self.maze_length_slider = widgets.IntSlider(min=30, max=100, value=50, description = "Set the Mazelength (in cm)!")
            
            row0 = HBox([clean_df_button, self.framerate_slider, self.likelihood_threshold_slider, self.maze_length_slider])
            box = VBox([row0])
            
            display(box)

            
        def get_processed_df(self, b):
            if self.clean_df == False:
                DLC_likelihood_threshold = self.likelihood_threshold_slider.value
                main.framerate[cam] = self.framerate_slider.value
                main.mazelength = self.maze_length_slider.value
                main.get_processed_dfs(main, DLC_likelihood_threshold, self.cam)
                self.clean_df = True
            else:
                print("Dataframe already cleaned")
            
    class select_functions(main):
        def __init__(self):
            select_functions = widgets.Label(value="Select functions in which you're interested in")
            self.immobility_check = widgets.Checkbox(
                        value=False,
                        description='Immobility',
                        disabled=False,
                        indent=False)
            self.gait_disruption_check = widgets.Checkbox(
                        value=False,
                        description='Gait Disruption',
                        disabled=False,
                        indent=False)
            self.freezing_check = widgets.Checkbox(
                        value=False,
                        description='Freezing',
                        disabled=False,
                        indent=False)
            self.Piro_Inken_check = widgets.Checkbox(
                        value=False,
                        description='Funktionen für Inken',
                        disabled=False,
                        indent=False)
            
            self.l_checkboxes = [self.immobility_check, self.gait_disruption_check, self.freezing_check, self.Piro_Inken_check]

            confirm_selection_button = widgets.Button(description = "Confirm Selection")
            confirm_selection_button.on_click(self.confirm_selection)

            col0 = VBox([select_functions, self.immobility_check, self.gait_disruption_check, self.freezing_check, self.Piro_Inken_check])
            col1 = VBox([confirm_selection_button])
            box = HBox([col0, col1])
            display(box)

        def confirm_selection(self, b):
            l_selected_functions_values = [checkbox.value for checkbox in self.l_checkboxes]
            l_selected_functions_keys = [checkbox.description for checkbox in self.l_checkboxes]
            dict_selected_functions = {key:value for key,value in zip(l_selected_functions_keys,l_selected_functions_values)}
            main.execute_functions(main, dict_selected_functions)

In [93]:
main()

'C:/Users/kobel/Documents/Medizin/Doktorarbeit/Coding/Dummy Data//'

Output(layout=Layout(height='600px'))

In [60]:
l_bodypartss = [['TailBase', 'TailEnd', 'TailMiddle', 'TailQuarterAnt', 'TailQuarterPost', 'HindPawRight', 'HindPawRightPlanted', 'HindPawLeft', 'HindPawLeftPlanted', 'ForePawRight', 'ForePawRightPlanted', 'ForePawLeft', 'ForePawLeftPlanted', 'Snout', 'Genitals', 'EarRight', 'EarLeft']]
print(l_bodypartss)

[['TailBase', 'TailEnd', 'TailMiddle', 'TailQuarterAnt', 'TailQuarterPost', 'HindPawRight', 'HindPawRightPlanted', 'HindPawLeft', 'HindPawLeftPlanted', 'ForePawRight', 'ForePawRightPlanted', 'ForePawLeft', 'ForePawLeftPlanted', 'Snout', 'Genitals', 'EarRight', 'EarLeft']]


In [65]:
class stats:
    pass

In [66]:
class subjects:
    
    def __init__(self, subject_ID):
        self.subject_ID = subject_ID
        

In [118]:
#ersten frame bei setmazecorners schon zeigen bevor man Koordinaten aktualisiert (Frame von vornherein plotten)
#IntSlider durch IntText? ersetzen, weil schneller
#prüfen, dass wirklich überprüft wird, ob alle Dateien da sind 
#BC-Gui mit label abgrenzen